In [1]:
! pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas 

In [3]:
import pandas as pd
import spacy
import langchain

print("All packages ready 🚀")


All packages ready 🚀


In [4]:
import pandas as pd
import numpy as np

print("Imports successful")


Imports successful


In [5]:
import pandas as pd

DATA_PATH = "D:\contract-intelligence-system\data\legal_docs.csv"

df = pd.read_csv(DATA_PATH)

print("Dataset loaded successfully ✅")
print("Shape:", df.shape)

df.head()


<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\saiak\AppData\Local\Temp\ipykernel_26928\3521733811.py:3: SyntaxWarning: invalid escape sequence '\c'
  DATA_PATH = "D:\contract-intelligence-system\data\legal_docs.csv"


Dataset loaded successfully ✅
Shape: (21187, 5)


,Unnamed: 0,clause_text,clause_type,totalwords,totalletters
0,0,"Make any Investments, except:",investments,4.0,30.0
1,1,No more than 45% of the “value” (as defined i...,investments,76.0,460.0
2,2,"Make or hold any Investments, except:",investments,6.0,38.0
3,3,The SubAdviser is hereby authorized and direc...,investments,228.0,1474.0
4,4,"Make any advance, loan, extension of credit (...",investments,52.0,329.0


In [6]:
df = df.drop(columns=["Unnamed: 0"])

df.head()


,clause_text,clause_type,totalwords,totalletters
0,"Make any Investments, except:",investments,4.0,30.0
1,No more than 45% of the “value” (as defined i...,investments,76.0,460.0
2,"Make or hold any Investments, except:",investments,6.0,38.0
3,The SubAdviser is hereby authorized and direc...,investments,228.0,1474.0
4,"Make any advance, loan, extension of credit (...",investments,52.0,329.0


In [7]:
df["clause_type"].value_counts()


clause_type
interest                  1010
base-salary               1010
ownership_of_shares       1000
payment                   1000
taxes                     1000
investment-company-act    1000
compensation              1000
investments               1000
capitalization             930
loans                      920
Definitions                890
Headings                   860
WHEREAS                    730
Entire                     670
Assignment                 630
Counterparts               630
Representations            610
Termination                590
Severability               580
NOW                        540
Miscellaneous              530
Insurance                  470
Indemnification            370
dividends                  360
Confidentiality            360
Governing                  330
Notices                    320
board                      250
esop                       223
payment_terms              220
grant                      220
financing                  

In [8]:
df["clause_text"].iloc[0]


' Make any Investments, except:'

In [9]:
df["clause_type"].value_counts()


clause_type
interest                  1010
base-salary               1010
ownership_of_shares       1000
payment                   1000
taxes                     1000
investment-company-act    1000
compensation              1000
investments               1000
capitalization             930
loans                      920
Definitions                890
Headings                   860
WHEREAS                    730
Entire                     670
Assignment                 630
Counterparts               630
Representations            610
Termination                590
Severability               580
NOW                        540
Miscellaneous              530
Insurance                  470
Indemnification            370
dividends                  360
Confidentiality            360
Governing                  330
Notices                    320
board                      250
esop                       223
payment_terms              220
grant                      220
financing                  

In [10]:
# Define clause type mappings

TERMINATION_TYPES = [
    "Termination"
]

LIABILITY_TYPES = [
    "Indemnification",
    "Insurance"
]

CONFIDENTIALITY_TYPES = [
    "Confidentiality",   # if exists
    "Non-Disclosure",    # if exists
    "Secrecy"            # fallback (dataset dependent)
]


In [11]:
# Filter dataset to only relevant clause types

filtered_df = df[
    df["clause_type"].isin(
        TERMINATION_TYPES + LIABILITY_TYPES + CONFIDENTIALITY_TYPES
    )
].copy()

print("Filtered shape:", filtered_df.shape)
filtered_df["clause_type"].value_counts()


Filtered shape: (1790, 4)


clause_type
Termination        590
Insurance          470
Indemnification    370
Confidentiality    360
Name: count, dtype: int64

In [12]:
def normalize_clause_type(ct):
    if ct in TERMINATION_TYPES:
        return "Termination"
    elif ct in LIABILITY_TYPES:
        return "Liability"
    elif ct in CONFIDENTIALITY_TYPES:
        return "Confidentiality"
    else:
        return "Other"

filtered_df["normalized_clause_type"] = filtered_df["clause_type"].apply(normalize_clause_type)

filtered_df["normalized_clause_type"].value_counts()


normalized_clause_type
Liability          840
Termination        590
Confidentiality    360
Name: count, dtype: int64

In [13]:
! pip install langchain-openai



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
print("LLM initialized successfully")

LLM initialized successfully


In [15]:
! pip install langchain-huggingface huggingface_hub



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

print("Hugging Face integration ready 🚀")


Hugging Face integration ready 🚀


In [17]:
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.3,
    huggingfacehub_api_token="YOUR_HUGGINGFACE_TOKEN"
)

print("LLM initialized 🚀")


LLM initialized 🚀


In [18]:
from langchain_core.prompts import PromptTemplate

risk_prompt = PromptTemplate(
    input_variables=["clause_text", "clause_type"],
    template="""
You are a legal risk analyst.

Analyze the following {clause_type} clause and explain:
1. What the clause means
2. Who is affected
3. What risk it creates

Explain in simple, clear language for a non-legal audience.

Clause:
{clause_text}
"""
)

print("Risk prompt ready 🧠")


Risk prompt ready 🧠


In [19]:
sample = filtered_df.sample(1).iloc[0]

print("Clause Type:", sample["normalized_clause_type"])
print("\nClause Text:\n")
print(sample["clause_text"])


Clause Type: Confidentiality

Clause Text:

Each Investor agrees that such Investor will keep confidential and will not disclose, divulge, or use for any purpose (other than to monitor its investment in the Company) any confidential information obtained from the Company pursuant to the terms of this Agreement (including notice of the Company’s intention to file a registration statement), unless such confidential information (a) is known or becomes known to the public in general (other than as a result of a breach of this Subsection 3.4 by such Investor), (b) is or has been independently developed or conceived by the Investor without use of the Company’s confidential information, or (c) is or has been made known or disclosed to the Investor by a third party without a breach of any obligation of confidentiality such third party may have to the Company; provided, however, that an Investor may disclose confidential information (i) to its attorneys, accountants, consultants, and other profe

In [ ]:
from langchain_core.messages import HumanMessage

risk_input = risk_prompt.format(
    clause_text=sample["clause_text"],
    clause_type=sample["normalized_clause_type"]
)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

# Base HF endpoint
hf_endpoint = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.3,
    huggingfacehub_api_token="YOUR_HUGGINGFACE_TOKEN"
)

# Wrap it as a CHAT model
llm = ChatHuggingFace(llm=hf_endpoint)

print("Chat Hugging Face LLM ready 🚀")

response = llm.invoke([
    HumanMessage(content=risk_input)
])

print("🧠 Risk Summary:\n")
print(response.content)


Chat Hugging Face LLM ready 🚀
🧠 Risk Summary:

 This confidentiality clause is a agreement between an Investor and a Company, stating that the Investor must keep any confidential information they receive from the Company a secret, and only use it for monitoring their investment in the Company. Here's a simple explanation:

1. What the clause means: The Investor is not allowed to share any confidential information they get from the Company with anyone else, except in certain specific situations. This includes information about the Company's plans to file a public registration statement. The Investor can only use this information to keep track of their investment in the Company.

2. Who is affected: This clause applies to each Investor who enters into an agreement with the Company. The Investor is bound by the confidentiality provisions and must keep the Company's information secret.

3. What risk it creates: The main risk created by this clause is the potential for breaches of confident

In [21]:
risk_prompt = PromptTemplate(
    input_variables=["clause_text", "clause_type"],
    template="""
You are a legal risk analyst.

Analyze the following {clause_type} clause and explain:
1. What the clause means
2. Who is affected
3. What risk it creates

Explain in simple, clear language for a non-legal audience.

Clause:
{clause_text}
"""
)


In [22]:
risk_input = risk_prompt.format(
    clause_text=sample["clause_text"],
    clause_type=sample["normalized_clause_type"]
)

response = llm.invoke([
    HumanMessage(content=risk_input)
])

print("🧠 Risk Summary:\n")
print(response.content)


🧠 Risk Summary:

 This confidentiality clause is a part of an investment agreement between an investor and a company. It means that the investor agrees to keep any confidential information they receive from the company secret, and not share it with anyone else without permission. This information could include things like financial data, business plans, or other sensitive information.

The clause affects both the investor and the company. The investor is bound by the confidentiality agreement and must keep the information confidential, while the company is the one sharing the confidential information.

The risk created by this clause is that if the investor breaches the confidentiality agreement and shares the information with others, it could harm the company. For example, if the investor shares confidential information with a competitor, it could give them an unfair advantage over the company. Additionally, if the investor shares the information publicly, it could negatively impact t

In [23]:
def assign_risk_severity(text, clause_type):
    text = text.lower()
    
    high_risk_keywords = [
        "unlimited", "indemnify", "terminate immediately",
        "sole discretion", "without notice", "all damages"
    ]
    
    medium_risk_keywords = [
        "may", "reasonable", "subject to", "material",
        "notice", "except"
    ]
    
    if any(word in text for word in high_risk_keywords):
        return "High"
    elif any(word in text for word in medium_risk_keywords):
        return "Medium"
    else:
        return "Low"


In [24]:
severity = assign_risk_severity(
    sample["clause_text"],
    sample["normalized_clause_type"]
)

print("⚠️ Risk Severity:", severity)


⚠️ Risk Severity: Medium


In [25]:
#random sample of 5 clauses for testing

from langchain_core.messages import HumanMessage

# Pick 5 random clauses
samples = filtered_df.sample(5)

for idx, row in samples.iterrows():
    print("=" * 80)
    print("📄 Clause Type:", row["normalized_clause_type"])
    print("\n📜 Clause Text:\n")
    print(row["clause_text"])
    
    # Format prompt
    risk_input = risk_prompt.format(
        clause_text=row["clause_text"],
        clause_type=row["normalized_clause_type"]
    )
    
    # LLM risk summary
    response = llm.invoke([
        HumanMessage(content=risk_input)
    ])
    
    print("\n🧠 Risk Summary:\n")
    print(response.content)
    
    # Risk severity
    severity = assign_risk_severity(
        row["clause_text"],
        row["normalized_clause_type"]
    )
    
    print("\n⚠️ Risk Severity:", severity)


📄 Clause Type: Termination

📜 Clause Text:

Notwithstanding whatever may be provided herein to the contrary, this Agreement may be terminated at any time, without payment of any penalty:

🧠 Risk Summary:

 1. This clause means that either party to the Agreement (the contract or agreement between two or more parties) has the right to end the agreement at any time, without giving a reason and without having to pay any financial penalty or compensation to the other party. In simpler terms, it allows either party to walk away from the agreement without any consequences, other than the termination itself.

2. This clause affects both parties to the Agreement. It gives each party the power to terminate the agreement unilaterally, meaning they can choose to end it on their own, without the involvement or consent of the other party.

3. The risk created by this clause is that either party may choose to terminate the agreement without cause, which could potentially disrupt the other party's pla

In [30]:
from langchain_core.messages import HumanMessage

# Manual input clause (demo purpose)
user_clause = """
The receiving party shall not disclose any confidential information received
under this agreement to any third party without prior written consent.
"""

user_clause_type = "Confidentiality"

risk_input = risk_prompt.format(
    clause_text=user_clause,
    clause_type=user_clause_type
)

response = llm.invoke([
    HumanMessage(content=risk_input)
])

severity = assign_risk_severity(user_clause, user_clause_type)

print("📄 Clause Type:", user_clause_type)
print("\n📜 Clause Text:\n", user_clause)
print("\n🧠 Risk Summary:\n", response.content)
print("\n⚠️ Risk Severity:", severity)


📄 Clause Type: Confidentiality

📜 Clause Text:
 
The receiving party shall not disclose any confidential information received
under this agreement to any third party without prior written consent.


🧠 Risk Summary:
  1. This confidentiality clause means that the receiving party (the person or entity that is receiving information under this agreement) is obligated to keep any confidential information they receive from the disclosing party (the person or entity that is sharing the information) a secret. They are not allowed to share this information with anyone else, including other businesses or individuals, without first getting written permission from the disclosing party.

2. This clause affects both the disclosing party and the receiving party. The disclosing party is sharing sensitive information and wants to ensure it remains confidential. The receiving party is under a legal obligation to keep the information secret and not share it with anyone without permission.

3. The risk cr

In [27]:
samples = filtered_df.sample(5)


In [28]:
samples = filtered_df.sample(2)


In [29]:
try:
    response = llm.invoke([HumanMessage(content=risk_input)])
    print("\n🧠 Risk Summary:\n")
    print(response.content)
except Exception as e:
    print("\n⚠️ LLM call timed out. Skipping this clause.")



🧠 Risk Summary:

 1. This clause means that either you or the other party in the agreement have the right to end the agreement suddenly and without warning if the other party fails to meet a significant or important obligation under the contract.

2. This clause affects both parties in the agreement. It gives each party the power to terminate the agreement if the other party does not meet a major obligation.

3. The risk created by this clause is that either party could suddenly terminate the agreement without warning, even if the breach of the obligation was minor or easily remedied. This could cause disruption, financial loss, or other negative consequences for the terminated party. It's important for both parties to understand their obligations under the contract and work to meet them to avoid the risk of an unexpected termination.
